In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import refinitiv.dataplatform as rdp
import eikon as ek

In [2]:
df_2 = pd.read_csv('emissions.csv')
df_new2 = df_2.sample(n=300)
df_new2 = df_new2.reset_index(drop=True)
df_new2

,Company,E1-2018,E1-2019,E1-2020,E1-2021,E2L-2018,E2L-2019,E2L-2020,E2L-2021,E2M-2018,E2M-2019,E2M-2020,E2M-2021
0,"Cardinal Health, Inc.",-,169000,153000,156100,-,350000,290000,265200,-,-,-,-
1,Genpact Ltd,4506,9286,7417,4508,-,-,-,-,106972,85997,44747,36426
2,GEA Group AG,39983,36711,30394,35077,36068,36768,34286,31504,-,32441,28488,15726
3,Ipsen,14750,18553,14916,15408,12450,11231,10421,9907,3470,5191,5377,3826
4,Kmart Australia Limited,-,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Brambles,49510,26650,28330,29410,40930,32800,30060,-,-,22230,15210,12950
296,Johnson Controls International plc,631752,642379,549358,451311,402455,386221,331236,300725,305578,264108,229809,212918
297,Ahold Delhaize,1816063,1699586,1579000,1728000,1949930,1925676,1820000,1748000,1707280,1771909,1457000,1099000
298,Lyft Inc,-,-,718,1022,-,-,8191,6167,-,-,1650,795


In [3]:
ek.set_app_key('58f10218ec2745df8b72a10d7f212b9428071259')
companies2 = df_new2['Company'].tolist()
for i, co in enumerate(companies2):
    companies2[i] = co.replace('Corporation', 'Corp')
    companies2[i] = co.replace('Incorporated', 'Inc')
    companies2[i] = co.replace('Limited', 'Ltd')
    companies2[i] = co.replace('Company', 'Co')
    companies2[i] = co.replace('Inc.', 'Inc')
    companies2[i] = co.replace('Corp.', 'Corp')
    companies2[i] = co.replace('Ltd.', 'Ltd')
    companies2[i] = co.replace(',', '')
    companies2[i] = co.replace('.', '')

df_tickers2 = pd.DataFrame(0, index = np.arange(len(companies2)), columns=['RIC'])


for i, co in enumerate(companies2):
    company_name = co
    screener_exp = 'SCREEN(U(IN(Equity(active,public,primary))),Contains(TR.CommonName,"{}"))'.format(company_name)
    try:
        company_ticker, err = ek.get_data(instruments=[screener_exp], fields = ['TR.RIC','TR.CommonName','TR.TickerSymbol'])
    except:
        pass
    print(company_ticker)
    if company_ticker is not None:
        df_tickers2['RIC'][i] = company_ticker['RIC'][0]
    else:
        df_tickers2['RIC'][i] = np.nan

df_tickers2

None
  Instrument  RIC Company Common Name Ticker Symbol
0        G.N  G.N         Genpact Ltd             G
  Instrument      RIC Company Common Name Ticker Symbol
0    G1AG.DE  G1AG.DE        GEA Group AG           G1A
  Instrument     RIC Company Common Name Ticker Symbol
0     IPN.PA  IPN.PA            Ipsen SA           IPN
None
  Instrument    RIC Company Common Name Ticker Symbol
0      ALV.N  ALV.N         Autoliv Inc           ALV
None
None
None
  Instrument      RIC Company Common Name Ticker Symbol
0    ABUJ.NS  ABUJ.NS  Ambuja Cements Ltd     AMBUJACEM
None
  Instrument       RIC                               Company Common Name  \
0    CARR.PA   CARR.PA                                      Carrefour SA   
1   CRFSA.IS  CRFSA.IS  CarrefourSA Carrefour Sabanci Ticaret Merkezi AS   

  Ticker Symbol  
0            CA  
1       CRFSA.E  
  Instrument     RIC Company Common Name Ticker Symbol
0     BNZL.L  BNZL.L           Bunzl plc          BNZL
None
  Instrument     RIC Compa

,RIC
0,NaN
1,G.N
2,G1AG.DE
3,IPN.PA
4,NaN
...,...
295,BXB.AX
296,JCI.N
297,AD.AS
298,LYFT.OQ


In [10]:
pd.set_option('display.max_rows', None)

df_fin2 = df_tickers2.merge(df_new2, left_index=True, right_index=True)
df_fin2.dropna(subset=['RIC'], inplace=True)
df_fin2 = df_fin2.reset_index(drop=True)
df_fin2

,RIC,Company,E1-2018,E1-2019,E1-2020,E1-2021,E2L-2018,E2L-2019,E2L-2020,E2L-2021,E2M-2018,E2M-2019,E2M-2020,E2M-2021
0,G.N,Genpact Ltd,4506,9286,7417,4508,-,-,-,-,106972,85997,44747,36426
1,G1AG.DE,GEA Group AG,39983,36711,30394,35077,36068,36768,34286,31504,-,32441,28488,15726
2,IPN.PA,Ipsen,14750,18553,14916,15408,12450,11231,10421,9907,3470,5191,5377,3826
3,ALV.N,Autoliv Inc,-,107000,98000,103000,-,280000,270000,290000,-,326000,315000,333000
4,ABUJ.NS,Ambuja Cements,14849220,14523738,13405629,16180247,539597,551219,537403,601907,18937,34717,88187,-
5,CARR.PA,Carrefour,1006784,827791,698695,613932,1555296,1005195,965101,814382,1443471,1032915,963598,869069
6,BNZL.L,Bunzl plc,99848,99193,90568,87125,31615,29594,27421,25043,29033,29835,26183,25025
7,DAN,"Yum China Holdings, Inc.",-,114720,135615,158887,-,1956072,1873757,2158856,-,-,-,-
8,BT.L,BT Group,184363,182565,171422,180779,795642,720878,624729,554243,114098,60649,262,155
9,SUZB3.SA,Suzano SA,-,2142411,2155103,2328336,-,71226,59532,137823,-,-,-,-


In [11]:
df_fin2['RIC'][13] = 'DAN'
df_fin2['RIC'][27] = 'EONGn.DE'
df_fin2.drop(index=29, inplace=True)
df_fin2.drop(index=33, inplace=True)
df_fin2.drop(index=44, inplace=True)
df_fin2.drop(index=46, inplace=True)

In [12]:
df_new_tickers = df_fin2[0:51]
df_new_tickers.reset_index(drop=True, inplace=True)

,RIC,Company,E1-2018,E1-2019,E1-2020,E1-2021,E2L-2018,E2L-2019,E2L-2020,E2L-2021,E2M-2018,E2M-2019,E2M-2020,E2M-2021
0,G.N,Genpact Ltd,4506,9286,7417,4508,-,-,-,-,106972,85997,44747,36426
1,G1AG.DE,GEA Group AG,39983,36711,30394,35077,36068,36768,34286,31504,-,32441,28488,15726
2,IPN.PA,Ipsen,14750,18553,14916,15408,12450,11231,10421,9907,3470,5191,5377,3826
3,ALV.N,Autoliv Inc,-,107000,98000,103000,-,280000,270000,290000,-,326000,315000,333000
4,ABUJ.NS,Ambuja Cements,14849220,14523738,13405629,16180247,539597,551219,537403,601907,18937,34717,88187,-
5,CARR.PA,Carrefour,1006784,827791,698695,613932,1555296,1005195,965101,814382,1443471,1032915,963598,869069
6,BNZL.L,Bunzl plc,99848,99193,90568,87125,31615,29594,27421,25043,29033,29835,26183,25025
7,DAN,"Yum China Holdings, Inc.",-,114720,135615,158887,-,1956072,1873757,2158856,-,-,-,-
8,BT.L,BT Group,184363,182565,171422,180779,795642,720878,624729,554243,114098,60649,262,155
9,SUZB3.SA,Suzano SA,-,2142411,2155103,2328336,-,71226,59532,137823,-,-,-,-


In [1]:
df3 = pd.read_csv('ticker_list.csv')

df_tick_merge = df_new_tickers[['RIC', 'Company']]
df_final_2 = pd.concat([df3, df_tick_merge], axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
df_final_2 = df_final_2[0:126]
df_final_2.drop(df_final_2['RIC'] == 'ADSK.O')
df_final_2.reset_index(drop=True, inplace=True)
df_final_2.to_excel(r'C:\Users\Evan Mc Garry\Desktop\FDS\FDS\GDS\ticker_list_final.xlsx', index=False)

NameError: name 'pd' is not defined